In [1]:
import pandas as pd, numpy as np
import os, glob

#os.chdir("C:/Users/jstelman/Git/chelsvig_urban_pesticides/src/")
os.chdir("C:/Git/chelsvig_urban_pesticides/src/")
from path_names import *
vvwm_path = os.path.join(os.path.abspath("../.."),"vvwm")
print(vvwm_path)

obs_path = os.path.join(main_path, "observed_data", "SURF_water_placer_bifenthrin.csv")
print(obs_path)

outfalls = ['outfall_31_26', 'outfall_31_28', 'outfall_31_29', 'outfall_31_35', 
            'outfall_31_36', 'outfall_31_38', 'outfall_31_42']

def output_path(of, i):
    op_path = os.path.join(vvwm_path, of, ("input_" + str(i)), "output_NPlesant_Custom_parent_daily.csv")
    return(op_path)

main_path   C:\Git\chelsvig_urban_pesticides
dir_path    C:\Git\chelsvig_urban_pesticides\probabilistic_python
exe_path    C:\Git\chelsvig_urban_pesticides\probabilistic_python\exe
swmm_path   C:\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm
inp_path    C:\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\NPlesantCreek.inp
bin_path    C:\Git\chelsvig_urban_pesticides\probabilistic_python\input\swmm\NPlesantCreek.out
vvwm_path   C:\Git\chelsvig_urban_pesticides\probabilistic_python\input\vvwm
wet_path    C:\Git\chelsvig_urban_pesticides\probabilistic_python\weather
C:\Git\vvwm
C:\Git\chelsvig_urban_pesticides\observed_data\SURF_water_placer_bifenthrin.csv


In [2]:
# Import Observed Data
obs_data = pd.read_csv(obs_path, usecols=["Concentration..ppb.","Level_of_quantification..ppb.",
                                         "Method_detection_level..ppb.", "Sample_date", "Site_code"],
                      parse_dates=["Sample_date"])

# do Proxy
for idx, row in obs_data[obs_data['Concentration..ppb.']==0].iterrows():
    obs_data.loc[idx,'Concentration..ppb.'] = min([row['Level_of_quantification..ppb.'],
                                                   row['Method_detection_level..ppb.']])/2

# drop columns
obs_data = obs_data.drop(labels = ["Level_of_quantification..ppb.","Method_detection_level..ppb."], axis = 1)
obs_data.head()

,Concentration..ppb.,Sample_date,Site_code
0,0.00235,2015-06-08,31_26
1,0.00235,2010-06-24,31_44
2,0.00235,2009-05-06,31_44
3,0.00235,2010-09-03,31_44
4,0.00035,2016-10-14,31_26


## For 112 simulations

In [3]:
inputs = [os.path.basename(name) for name in glob.glob(os.path.join(vvwm_path, "outfall_31_26", "input_*"))]
def output_path2(of, ip):
    op_path = os.path.join(vvwm_path, of, ip, "output_NPlesant_Custom_parent_daily.csv")
    return(op_path)

In [4]:
# Now do this as a loop for simulations 1-n, and save their n NSEs
NSEs = []

for i in inputs:
    
    sim1_of26_data = pd.read_csv(output_path2("outfall_31_26", i), usecols = [1], skiprows=5, names = ["davg_bif_conc"])*1000000
    sim1_of26_data['Sample_date'] = pd.date_range(start='1/1/2009', periods=3287, freq='D')
    sim1_of26_data['Site_code'] = '31_26'
    
    comp_data_ipi = sim1_of26_data.merge(obs_data, how = "inner", on = ['Sample_date','Site_code'])
    
    for of in outfalls[1:]:
        dat = pd.read_csv(output_path2(of, i), usecols = [1], skiprows=5, names = ["davg_bif_conc"])*1000000
        dat['Sample_date'] = pd.date_range(start='1/1/2009', periods=3287, freq='D')
        dat['Site_code'] = of[-5:]
        comp_data_ipi = comp_data_ipi.append(dat.merge(obs_data, how = "inner", on = ['Sample_date','Site_code']), ignore_index=True)
        
    NSEi = 1 - ((sum(comp_data_ipi['davg_bif_conc'] - comp_data_ipi['Concentration..ppb.'])**2)/
                (sum(comp_data_ipi['Concentration..ppb.'] - sum(comp_data_ipi['Concentration..ppb.'])/
                     len(comp_data_ipi['Concentration..ppb.']))**2))
    NSEs.append(NSEi)

In [5]:
NSEs
# Wow, these are all terrrrible

[-4.066021452443556e+54,
 -3.8599171755554547e+55,
 -2.206865728774752e+55,
 -2.2277674345547663e+55,
 -2.805538927482503e+54,
 -3.114161955127447e+54,
 -2.3069616075356395e+55,
 -2.9590266780416298e+54,
 -5.06800584567615e+54,
 -3.844067985368535e+54,
 -1.6847766751798336e+54,
 -7.978032285166373e+54,
 -1.8163983425407886e+55,
 -7.285010369869462e+54,
 -7.60846173096257e+54,
 -1.5446682050824052e+55,
 -1.0603705802402675e+55,
 -1.482395214375138e+55,
 -2.8490193639871263e+55,
 -3.4291274237991697e+55,
 -2.880486167015523e+55,
 -3.6680463343217095e+55,
 -2.2470001060234823e+55,
 -2.3895474501135302e+55,
 -7.020247123497176e+54,
 -6.560355149031143e+55,
 -9.993974439664223e+54,
 -2.478859552288944e+55,
 -2.389485197758379e+55,
 -2.49074600871382e+55,
 -5.49295390944757e+54,
 -1.8077580119808113e+55,
 -3.471613548673933e+54,
 -2.369169900570354e+55,
 -1.3245177930333755e+55,
 -2.7091995670743287e+54,
 -1.5903039872501029e+55,
 -1.5481287591584148e+55,
 -1.4423894379556296e+54,
 -8.744759

In [6]:
def get_subseted_ranges(acc_rate, vvwm = False):
    # how many will be in acceptanct subset?
    n_acc = int(np.ceil(len(inputs)*acc_rate))
    # which ones will they be?
    sbst = [int(inputs[i][6:])-1 for i in sorted(range(len(NSEs)), key=lambda i: NSEs[i])[-n_acc:]]
    if not vvwm:
        # lhs_path = os.path.join(dir_path, "io", "lhs_sampled_params.csv")
        lhs_path = os.path.join(vvwm_path, "lhs_params", "lhs_sampled_params.csv")
        old_param_ranges = pd.read_csv(os.path.join(dir_path, "input", "lhs", "lhs_param_ranges.csv"), index_col=0)
        # new_path = os.path.join(dir_path, "input", "lhs", "lhs_param_ranges_subset.csv")
        new_path = os.path.join(vvwm_path, "lhs_params", "lhs_param_ranges_" + str(round(acc_rate*100)) + "acc_rate.csv")
    else:
        # lhs_path = os.path.join(dir_path, "io", "lhs_sampled_params_vvwm.csv")
        lhs_path = os.path.join(vvwm_path, "lhs_params", "lhs_sampled_params_vvwm.csv")
        old_param_ranges = pd.read_csv(os.path.join(dir_path, "input", "lhs", "lhs_param_ranges_vvwm.csv"), index_col=0)
        # new_path = os.path.join(dir_path, "input", "lhs", "lhs_param_ranges_vvwm_subset.csv")
        new_path = os.path.join(vvwm_path, "lhs_params", "lhs_param_ranges_vvwm_" + str(round(acc_rate*100)) + "acc_rate.csv")

    lhsp = pd.read_csv(lhs_path, index_col=0)

    # subset to just those pertaining to the winners
    subset_lhs = lhsp.iloc[sbst]

    # make a new df for the ranges 
    # start by copying the old one
    new_lhs_ranges = old_param_ranges.copy()
    for idx, row in new_lhs_ranges.iterrows():
        new_lhs_ranges.loc[idx,'Min'] = min(subset_lhs[idx])
        new_lhs_ranges.loc[idx,'Max'] = max(subset_lhs[idx])
        new_lhs_ranges.loc[idx,'Range'] = max(subset_lhs[idx]) - min(subset_lhs[idx])
    # save this as into a file in the folder with the old one
    new_lhs_ranges.to_csv(new_path)
    print(old_param_ranges - new_lhs_ranges)
    return(new_lhs_ranges)

## Get the lhs_parameters data

### First the SWMM side

In [7]:
# get details of top 25% of Nash-Sutcliffe Efficiencies, in this case, 28
swmm_25 = get_subseted_ranges(.25, False)
# get details of top 10% of Nash-Sutcliffe Efficiencies, in this case, 12
swmm_10 = get_subseted_ranges(.1, False)
# get details of top 5% of Nash-Sutcliffe Efficiencies, in this case, 6
swmm_05 = get_subseted_ranges(.05, False)

                Min       Max     Range
Parameter                              
NImperv   -0.000005  0.001613  0.001618
NPerv     -0.004833  0.014007  0.018840
SImperv   -0.006261  0.016388  0.022650
SPerv     -0.021626  0.099428  0.121055
PctZero   -0.339498  3.234924  3.574422
MaxRate   -4.514092  0.256934  4.771027
MinRate   -0.019708  0.569391  0.589099
Decay     -0.103845  0.011330  0.115175
DryTime   -0.073133  0.248329  0.321462
Por       -0.001315  0.004896  0.006211
FC        -0.022395  0.002664  0.025059
WP        -0.003706  0.001521  0.005227
Ksat      -0.365326  0.787194  1.152520
Rough     -0.001626  0.000847  0.002474
Kdecay    -0.003047  0.011906  0.014953
BCoeff2   -0.034073  0.995869  1.029942
WCoeff2   -0.001681  0.007907  0.009588
                 Min       Max      Range
Parameter                                
NImperv    -0.000005  0.001875   0.001880
NPerv      -0.004833  0.029170   0.034003
SImperv    -0.281974  0.212431   0.494405
SPerv      -0.021626  0.459815

### Then the vvwm side

In [8]:
# get details of top 25% of Nash-Sutcliffe Efficiencies, in this case, 28
vvwm_25 = get_subseted_ranges(.25, True)
# get details of top 10% of Nash-Sutcliffe Efficiencies, in this case, 12
vvwm_10 = get_subseted_ranges(.1, True)
# get details of top 5% of Nash-Sutcliffe Efficiencies, in this case, 6
vvwm_05 = get_subseted_ranges(.05, True)

                      Min        Max       Range
Parameter                                       
kd            -119.172048  63.055218  182.227266
aer_aq         -12.336884  10.080396   22.417281
aer_aq_temp     -0.108423   0.049826    0.158249
anae_aq        -20.214437  34.270607   54.485043
anae_aq_temp    -0.380581   0.083237    0.463818
photo           -9.735126   9.533761   19.268887
rflat            0.000000   0.000000    0.000000
hydro           -5.491458   0.939632    6.431091
sol             -0.056822   0.001617    0.058438
benthic_depth   -0.052151   0.057981    0.110132
porosity        -0.054610   0.002192    0.056803
bulk_density    -0.060625   0.002650    0.063275
froc2           -0.000780   0.000453    0.001232
doc2            -1.003487   2.109367    3.112854
bnmas           -0.175594   0.062807    0.238401
sused           -2.715215   2.438923    5.154138
chl             -0.009083   0.029792    0.038875
froc1           -0.003129   0.006383    0.009512
doc1            -0.0